In [1]:
import sys
sys.path.append('..')
sys.path.append('./sae_vis/sae_vis')
import os
from tqdm import tqdm
import torch
from transformer_lens import HookedTransformer
from sae_training.sparse_autoencoder import SparseAutoencoder
from sae_training.activations_store import ActivationsStore
from feature_viz.sae_vis.sae_vis.data_fetching_fns import get_feature_data
from feature_viz.sae_vis.sae_vis.data_storing_fns import FeatureVizParams


/Users/tz20913/miniconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/tz20913/miniconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/tz20913/miniconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


ModuleNotFoundError: No module named 'sae_vis.model_fns'

In [3]:
sae_path = '../artifacts/sparse_autoencoder_gelu-1l_blocks.0.hook_resid_mid_16384:v4/final_sparse_autoencoder_gelu-1l_blocks.0.hook_resid_mid_16384.pt'
hook_point = 'resid_mid'
hook_layer = 0
feature_idx = 623
n_batches_to_sample_from = 2**12
n_prompts_to_select = 4096 * 6

In [ ]:
sae = SparseAutoencoder.load_from_pretrained(sae_path)
model = HookedTransformer.from_pretrained(sae.cfg.model_name)
model.to(sae.device)


Loaded pretrained model gelu-1l into HookedTransformer
Moving model to device:  mps


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resi

In [ ]:

tokens_path = f"tokens_{n_prompts_to_select}.pt"
# if the tokens are already on disk, load them
if os.path.exists(tokens_path):
    tokens = torch.load(tokens_path)
else: # otherwise load them from HuggingFace
    activation_store = ActivationsStore(sae.cfg, model)
    all_tokens_list = []
    for _ in tqdm(range(n_batches_to_sample_from)):
        batch_tokens = activation_store.get_batch_tokens()
        batch_tokens = batch_tokens[torch.randperm(batch_tokens.shape[0])][
            : batch_tokens.shape[0]
        ]
        all_tokens_list.append(batch_tokens)

    all_tokens = torch.cat(all_tokens_list, dim=0)
    all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]
    tokens = all_tokens[:n_prompts_to_select]

    torch.save(tokens, tokens_path)


In [ ]:
feature_viz_params = FeatureVizParams(include_left_tables=False)
feature_data = get_feature_data(
    encoder = sae,
    encoder_B = None,
    model = model,
    tokens = tokens,
    fvp = feature_viz_params,
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x512 and 2048x512)